No processo de **backpropagation** é necessário calcular o valor do **gradiente**, utilizando das **derivadas**. Por isso, é importante que nós consigamos determinar qual o valor esperado da derivada de algumas funções.

Vale lembrar que a derivada de uma função é calculada da seguinte forma:

\begin{align}
f'(x) = \frac{f(x+h) - f(x)}{h}
\end{align}

Quando se trata de uma operação de soma, como a seguir:

\begin{align}
A = b + c
\end{align}

Nós podemos inferir a derivada da seguinte maneira:

\begin{align}
\frac{dA}{db} &= \frac{((b + h) + c) - (b + c)}{h} \\
&= \frac{b + h + c - b - c}{h} \\
&= \frac{h}{h} \\
&= 1
\end{align}

Ou seja, quando se trata de uma **soma**, a derivada é sempre $1$.

Quando se trata de uma operação de multiplicação, como a seguir:

\begin{align}
A = b \times c
\end{align}

Nós podemos inferir a derivada da seguinte maneira:

\begin{align}
\frac{dA}{db} &= \frac{((b + h) \times c) - (b \times c)}{h} \\
&= \frac{b \times c + h \times c - b \times c}{h} \\
&= \frac{h \times c}{h} \\
&= c
\end{align}

Ou seja, quando se trata de uma **multiplicação**, a derivada sempre será o outro termo da função.

Quando se trata de $A^b$, nós temos que a derivada será $b \times A^{b-1}$, pela **Regra do Tombo**.

Quando se trata de uma função exponencial natural $e^x$, a derivada é igual à própria função.

Também será necessário fazer o uso de uma função de ativação e, neste caso, a função utilizada é a **Tangente Hiperbólica**:

\begin{align}
\frac{e^{2x} - 1}{e^{2x} + 1}
\end{align}

A derivada dessa função é $1 − tanh^2(x)$.

Além disso, é importante saber também que, pela **Regra da Cadeia**, nós temos que a derivada de $f(g(x))$ é $f'(g(x)) \times g'(x)$ e será dessa forma que nós conseguiremos calcular o *acúmulo* do gradiente.

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

A fim de visualizar melhor o que acontece em cada passo do **backpropagation**, nós vamos precisar da classe criada a seguir, baseada nos cálculos de derivadas mostradas anteriormente.

In [1]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  #Definição da operação de soma
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    #Como visto anteriormente, a derivada de uma soma é 1. Logo, com a regra da cadeia, basta fazer um "repasse" do gradiente que está "chegando"
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  #Definição da operação de multiplicação
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    #Como visto anteriormente, a derivada de uma multiplicação é o "outro termo". Logo, com a regra da cadeia, basta fazer uma multiplicação entre o "outro termo" e o grandiente que está "chegando"
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  #Definição da operação de potenciação
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    #Como visto anteriormente, de acordo com a Regra do Tombo, a derivada de A^b deve ser b*A^b-1. Logo, com a regra da cadeia, basta multiplicar o resultado com o gradiente que está "chegando" 
    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  #Definição da operação de multiplicação com a inversão dos fatores
  def __rmul__(self, other):
    return self * other

  #Definição da operação de divisão como uma operação de potenciação
  def __truediv__(self, other):
    return self * other**-1

  #Definição da operação do negativo de um valor
  def __neg__(self):
    return self * -1
  
  #Definição da operação de subtração
  def __sub__(self, other):
    return self + (-other)

  #Definição da operação de adição com a inversão dos valores
  def __radd__(self, other):
    return self + other

  #Definição da função Tangente Hiperbólica
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    #Cálculo do gradiente com base na derivada da função e, com a regra da cadeia, no gradiente que está "chegando"
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  #Definição da função de exponencial natural
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    #Como visto anteriormente, a derivada dessa função é ela mesma, por isso, basta multiplicar o seu valor pelo gradiente que está "chegando"
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    
    return out
  
  #Função que orquestra o cálculo dos gradientes
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()